In [2]:
import pandas as pd
import matplotlib as plt

In [3]:
sells_df = pd.read_csv("Electronic_sales.csv")

# Задание 1

In [74]:
sells_df['Payment Method'] = sells_df['Payment Method'].astype('category')

In [80]:
payment_counts = sells_df.groupby(['Customer ID', 'Payment Method'], observed=True).size().reset_index(name='counts')

favorite_payment = payment_counts.loc[payment_counts.groupby('Customer ID')['counts'].idxmax()]

print(favorite_payment)

favorite_payment.head(2200)

       Customer ID Payment Method  counts
0             1000    Credit Card       1
2             1002           Cash       1
4             1003           Cash       1
5             1004    Credit Card       1
6             1005     Debit Card       1
...            ...            ...     ...
17144        19994  Bank Transfer       1
17145        19995    Credit Card       1
17146        19996  Bank Transfer       2
17149        19997  Bank Transfer       1
17150        19998  Bank Transfer       1

[12136 rows x 3 columns]


,Customer ID,Payment Method,counts
0,1000,Credit Card,1
2,1002,Cash,1
4,1003,Cash,1
5,1004,Credit Card,1
6,1005,Debit Card,1
...,...,...,...
3143,4270,Cash,1
3146,4271,Debit Card,2
3148,4272,Credit Card,1
3150,4274,Paypal,2


In [54]:
total_sells = sells_df["Total Price"].sum()
print(f"Общие траты: {total_sells:.2f}")

Общие траты: 63602668.37


In [53]:
total_sells_succsess = sells_df[sells_df["Order Status"] == "Completed"]["Total Price"].sum()
print(f"Общие траты (успешные): {total_sells_succsess:.2f}")

Общие траты (успешные): 42629615.57


In [ ]:
ts_by_client = sells_df.groupby("Customer ID")["Total Price"].sum()

ts_by_client_succsess = sells_df[sells_df["Order Status"] == "Completed"].groupby("Customer ID")["Total Price"].sum()

In [64]:
print("Обшие траты клиентов:")
ts_by_client

Обшие траты клиентов:


Customer ID
1000     6279.42
1002     5020.60
1003       41.50
1004       83.00
1005    11779.11
          ...   
19994    5504.87
19995    5394.56
19996   27296.78
19997    2528.26
19998     674.32
Name: Total Price, Length: 12136, dtype: float64

In [65]:
print("Обшие успешные траты клиентов:")
ts_by_client_succsess

Обшие успешные траты клиентов:


Customer ID
1000      741.09
1002     5020.60
1003       41.50
1004       83.00
1005    11779.11
          ...   
19990    9117.44
19991   12368.06
19995    5394.56
19996   12063.02
19998     674.32
Name: Total Price, Length: 9466, dtype: float64

In [69]:
a_total_additional_sells = sells_df["Add-on Total"].sum()

In [70]:
print(f"{a_total_additional_sells:.2f}")

1244896.96


In [ ]:
s_total_additional_sells = sells_df[sells_df["Order Status"] == "Completed"]["Add-on Total"].sum()

In [ ]:
print(f"{s_total_additional_sells:.2f}")

835595.24


In [71]:
a_total_additional_sells_by_client = sells_df.groupby("Customer ID")["Add-on Total"].sum()

In [72]:
s_total_additional_sells_by_client = sells_df[sells_df["Order Status"] == "Completed"].groupby("Customer ID")["Add-on Total"].sum()

In [96]:
first_final_df = sells_df["Customer ID"].drop_duplicates().to_frame().join(favorite_payment.set_index("Customer ID"), on="Customer ID", how='left').join(ts_by_client.rename("Total Sells"), on="Customer ID", how='left').join(ts_by_client_succsess.rename("Total Sells Success"), on="Customer ID", how='left').join(a_total_additional_sells_by_client.rename("Add-on Total Sells"), on="Customer ID", how='left').join(s_total_additional_sells_by_client.rename("Add-on Total Sells Success"), on="Customer ID", how='left')

In [97]:
first_final_df

,Customer ID,Payment Method,counts,Total Sells,Total Sells Success,Add-on Total Sells,Add-on Total Sells Success
0,1000,Credit Card,1,6279.42,741.09,66.30,26.09
2,1002,Cash,1,5020.60,5020.60,60.16,60.16
4,1003,Cash,1,41.50,41.50,35.56,35.56
5,1004,Credit Card,1,83.00,83.00,65.78,65.78
6,1005,Debit Card,1,11779.11,11779.11,75.33,75.33
...,...,...,...,...,...,...,...
19990,19994,Bank Transfer,1,5504.87,NaN,93.17,NaN
19991,19995,Credit Card,1,5394.56,5394.56,0.00,0.00
19992,19996,Bank Transfer,2,27296.78,12063.02,432.12,198.98
19998,19997,Bank Transfer,1,2528.26,NaN,101.34,NaN


# Задание 2

Если доход то нас не интересуют строки с отменённым заказом

In [7]:
succsess_sells_df = sells_df[sells_df["Order Status"] == "Completed"]

In [8]:
succsess_sells_df

,Customer ID,Age,Gender,Loyalty Member,Product Type,SKU,Rating,Order Status,Payment Method,Total Price,Unit Price,Quantity,Purchase Date,Shipping Type,Add-ons Purchased,Add-on Total
1,1000,53,Male,No,Tablet,SKU1002,3,Completed,Paypal,741.09,247.03,3,2024-04-20,Overnight,Impulse Item,26.09
2,1002,41,Male,No,Laptop,SKU1005,3,Completed,Credit Card,1855.84,463.96,4,2023-10-17,Express,NaN,0.00
3,1002,41,Male,Yes,Smartphone,SKU1004,2,Completed,Cash,3164.76,791.19,4,2024-08-09,Overnight,"Impulse Item,Impulse Item",60.16
4,1003,75,Male,Yes,Smartphone,SKU1001,5,Completed,Cash,41.50,20.75,2,2024-05-21,Express,Accessory,35.56
5,1004,41,Female,No,Smartphone,SKU1001,5,Completed,Credit Card,83.00,20.75,4,2024-05-26,Standard,"Impulse Item,Accessory",65.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19991,19995,69,Female,Yes,Laptop,LTP123,3,Completed,Credit Card,5394.56,674.32,8,2024-08-09,Same Day,NaN,0.00
19994,19996,27,Female,No,Smartphone,SMP234,5,Completed,Credit Card,3419.04,1139.68,3,2024-04-20,Same Day,NaN,0.00
19995,19996,27,Female,No,Smartphone,SMP234,4,Completed,Bank Transfer,6838.08,1139.68,6,2024-06-15,Expedited,NaN,0.00
19997,19996,27,Female,No,Headphones,HDP456,4,Completed,Bank Transfer,1805.90,361.18,5,2024-08-26,Standard,"Impulse Item, Extended Warranty, Accessory",198.98


In [9]:
by_shipping = succsess_sells_df.groupby("Shipping Type")["Total Price"].sum()

In [10]:
by_shipping

Shipping Type
Expedited     8430385.58
Express       5632093.35
Overnight     5887722.52
Same Day      8292376.50
Standard     14387037.62
Name: Total Price, dtype: float64

In [11]:
by_product_category = succsess_sells_df.groupby("Product Type")["Total Price"].sum()

In [12]:
by_product_category

Product Type
Headphones     2734651.00
Laptop         8365905.25
Smartphone    14407835.84
Smartwatch     9398591.23
Tablet         7722632.25
Name: Total Price, dtype: float64

In [13]:
by_addon_month_df = succsess_sells_df.groupby(succsess_sells_df["Purchase Date"].str.slice(0,7))["Add-on Total"].sum()

In [14]:
by_addon_month_df

Purchase Date
2023-09     5337.61
2023-10    26153.21
2023-11    24453.33
2023-12    22750.23
2024-01    93254.95
2024-02    80253.72
2024-03    84713.93
2024-04    82294.06
2024-05    89374.18
2024-06    84648.60
2024-07    88811.55
2024-08    87861.11
2024-09    65688.76
Name: Add-on Total, dtype: float64

In [ ]:
by_addons_quarter_df = (
    succsess_sells_df.groupby(
        succsess_sells_df["Purchase Date"].str.slice(0,4) 
        + " Q" + 
        ((succsess_sells_df["Purchase Date"].str.slice(5,7).astype(int)-1)//3 +1).astype(str))["Add-on Total"]
    .sum()
)

In [16]:
by_addons_quarter_df

Purchase Date
2023 Q3      5337.61
2023 Q4     73356.77
2024 Q1    258222.60
2024 Q2    256316.84
2024 Q3    242361.42
Name: Add-on Total, dtype: float64